<a href="https://colab.research.google.com/github/tmontaj/Text-AudioDatasets/blob/main/Librispeech/Audio_Text_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import inflect
import re

In [30]:
# threshold : int

In [31]:
def audio_cleaning_old(arr):
  i=0
  print(threshold)
  while i<len(arr):
      if arr[i]>=threshold:
        arr_new= arr[i:]
        break   
      i +=1
  i=len(arr_new)-1
  while i>0:
      if arr_new[i]>=threshold:
        arr_new= arr_new[0:i+1]
        break   
      i -=1
  return arr_new


In [32]:
def audio_cleaning(audio, threshold):
  """
  clean audio from starting and trailing silence 
  
  Arguments:
  audio -- audio as np.array or tf tensor
  threshold -- path to the file
  Returns:
  audio -- clean audio 
  """
  found_start = False
  found_tail  = False

  start = 0
  tail  = audio.shape[0]-1

  while True:
    if found_start and found_tail: break
    if start >= tail: break
    
    if not found_start and audio[start]<threshold:
      start+=1
    else: 
      found_start = True

    if not found_tail and audio[tail]<threshold:
      tail-=1
    else: 
      found_tail = True

  return audio[start:tail+1]

In [33]:
# audio_cleaning(audio=np.array([0,0,00,4,5,6,6,6,6,7,8,5,5,5]),
#                threshold=5)

# audio_cleaning(audio=np.array([0,0,00,4]),
#                threshold=5)

In [34]:
def _collapse_whitespace(text):
  return text.replace(" ", "")

In [35]:
def _lowercase(text):
  return text.lower()

In [36]:
def _remove_commas(text):
  return text.replace(',', '')

In [37]:
def _remove_symbols(text):
  text = re.sub(r'[^a-zA-Z0-9 \'.,]', '', text)
  return text

In [38]:
_remove_symbols("text;.,/\ojn'\"~`!@#$$%^&*()_+{}<>?|")

"text.,ojn'"

In [39]:
_inflect = inflect.engine()
_decimal_number_re = re.compile(r'([0-9]+\.[0-9]+)')
_pounds_re = re.compile(r'£([0-9\,]*[0-9]+)')
_dollars_re = re.compile(r'\$([0-9\.\,]*[0-9]+)')
_ordinal_re = re.compile(r'[0-9]+(st|nd|rd|th)')
_number_re = re.compile(r'[0-9]+')

In [40]:
def _expand_decimal_point(text):
  return text.group(1).replace('.', ' point ')

In [41]:
def _expand_dollars(text):
  match = text.group(1)
  parts = match.split('.')
  if len(parts) > 2:
    return match + ' dollars'  # Unexpected format
  dollars = int(parts[0]) if parts[0] else 0
  cents = int(parts[1]) if len(parts) > 1 and parts[1] else 0
  if dollars and cents:
    dollar_unit = 'dollar' if dollars == 1 else 'dollars'
    cent_unit = 'cent' if cents == 1 else 'cents'
    return '%s %s, %s %s' % (dollars, dollar_unit, cents, cent_unit)
  elif dollars:
    dollar_unit = 'dollar' if dollars == 1 else 'dollars'
    return '%s %s' % (dollars, dollar_unit)
  elif cents:
    cent_unit = 'cent' if cents == 1 else 'cents'
    return '%s %s' % (cents, cent_unit)
  else:
    return 'zero dollars'

In [42]:
def _expand_ordinal(text):
  return _inflect.number_to_words(text.group(0))

In [43]:
def _expand_number(text):
  num = int(text.group(0))
  if num > 1000 and num < 3000:
    if num == 2000:
      return 'two thousand'
    elif num > 2000 and num < 2010:
      return 'two thousand ' + _inflect.number_to_words(num % 100)
    elif num % 100 == 0:
      return _inflect.number_to_words(num // 100) + ' hundred'
    else:
      return _inflect.number_to_words(num, andword='', zero='oh', group=2).replace(', ', ' ')
  else:
    return _inflect.number_to_words(num, andword='')

In [44]:
def _normalize_numbers(text):
  text = re.sub(_pounds_re, r'\1 pounds', text)
  text = re.sub(_dollars_re, _expand_dollars, text)
  text = re.sub(_decimal_number_re, _expand_decimal_point, text)
  text = re.sub(_ordinal_re, _expand_ordinal, text)
  text = re.sub(_number_re, _expand_number, text)
  return text

In [45]:
# List of (regular expression, replacement) pairs for abbreviations:
_abbreviations = [(re.compile('\\b%s\\.' % x[0], re.IGNORECASE), x[1]) for x in [
  ('mrs', 'misess'),
  ('mr', 'mister'),
  ('dr', 'doctor'),
  ('st', 'saint'),
  ('co', 'company'),
  ('jr', 'junior'),
  ('maj', 'major'),
  ('gen', 'general'),
  ('drs', 'doctors'),
  ('rev', 'reverend'),
  ('lt', 'lieutenant'),
  ('hon', 'honorable'),
  ('sgt', 'sergeant'),
  ('capt', 'captain'),
  ('esq', 'esquire'),
  ('ltd', 'limited'),
  ('col', 'colonel'),
  ('ft', 'fort'),
]]


In [46]:
def _expand_abbreviations(text):
  for regex, replacement in _abbreviations:
    text = re.sub(regex, replacement, text)
  return text

In [47]:
# print(normalize_numbers("I have eaten 5 apples and brought them for $200.90. They were 2kg"))

In [48]:
def clean_text(text, remove_comma=True):
  """
  clean text from symbols, remove (some) abbreviations, lowercase, and normaize numbers  
  
  Arguments:
  text -- text to clean
  remove_comma -- flag to remove commas
  Returns:
  text -- cleaned text
  """
  text = _lowercase(text)
  text = _normalize_numbers(text)
  text = _expand_abbreviations(text)
  text = _remove_symbols(text)
  if remove_comma:
    text = _remove_commas(text)
  return text

In [49]:
# print(clean_text("I have eaten 5 kg of apples in two days !! Can't believe they cost $3000.50!"))

i have eaten five kg of apples in two days  can't believe they cost three thousand dollars fifty cents
